# Example: SMIRNOFF99Frosst is not matching alpha-cyclodextrin bond types

In [1]:
import parmed as pmd
from openeye.oechem import *
from openforcefield.typing.engines.smirnoff import *

In [2]:
def load_mol2(filename, name=None, add_tripos=True):
    """
    Converts a `mol2` file to an `OEMol` object.
    Parameters
    ----------
    filename : str
        MOL2 file
    name : str
        Residue name
    add_tripos : bool
        Whether to add Tripos atom names to the file

    Returns
    -------
    openeye.oechem.OEMol

    """
    ifs = oemolistream()
    molecules = []
    if not ifs.open(filename):
        print(f'Unable to open {filename} for reading...')
    for mol in ifs.GetOEMols():
        if add_tripos:
            OETriposAtomNames(mol)
        if name:
            mol.SetTitle(name)
        # Add all the molecules in this file to a list, but only return the first one.
        molecules.append(OEMol(mol))
    return molecules[0]

In [3]:
def create_host_topology(components, host_resname):
    """Return the topology components belonging the host only.
    
    Parameters:
    ----------
    components : parmed.topology.components
        ParmEd topology components, split by molecule
    host_resname : str
        Residue name of the host molecule (no colon)
    Returns
    -------
    pmd.Structure
        A ParmEd structure containing just the host and guest molecule
    """
    topology = pmd.Structure()
    for component in components:
        # Check the first residue of each component becuase there may be multiple residues in each component, but they shoudl all have the same residue name.hash
        if component[0].residues[0].name == host_resname.upper():
            topology += component[0]
    return topology

In [4]:
def split_topology(file_name):
    """Split a file into component topology using ParmEd.
    
    Parameters:
    ----------
    file_name : str
        Structure file
    """

    topology = pmd.load_file(file_name)
    return topology.split()

## Try with GAFF atom types

In [5]:
host = load_mol2('generated/a-bam-p/MGO.mol2')
components = split_topology(file_name='generated/a-bam-p/full.pdb')
host_topology = create_host_topology(components, host_resname='MGO')

In [6]:
host.GetMaxAtomIdx()

126

In [7]:
host_topology

<Structure 126 atoms; 6 residues; 132 bonds; NOT parametrized>

In [8]:
ff = ForceField('forcefield/smirnoff99Frosst.ffxml') 
system = ff.createSystem(host_topology.topology, [host],
                         nonbondedCutoff=1.1*unit.nanometer, 
                         ewaldErrorTolerance=1e-4
                         )


In [9]:
host_structure = pmd.openmm.topsystem.load_topology(host_topology.topology, system, host_topology.positions)

In [10]:
pmd.tools.printBonds(host_structure)

             Atom 1              Atom 2       R eq   Frc Cnst   Distance     Energy
      1   C1 (   1)       2   H1 (   2)     1.0900   340.0000     1.1077     0.1067
      1   C1 (   1)       3   O1 (   3)     4.0000  1000.0000     1.4676  6412.9044
      1   C1 (   1)       4   C2 (   4)     1.5260   310.0000     1.5259     0.0000
      1   C1 (   1)      16   O5 (  16)     4.0000  1000.0000     1.4738  6381.4472
      3   O1 (   3)      33   C4 (  33)     4.0000  1000.0000     1.4655  6423.7720
      4   C2 (   4)       5   H2 (   5)     1.0900   340.0000     1.0934     0.0040
      4   C2 (   4)       6   O2 (   6)     1.3700   320.0000     1.4372     1.4435
      4   C2 (   4)       8   C3 (   8)     1.5260   310.0000     1.5140     0.0446
      6   O2 (   6)       7  HO2 (   7)     4.0000  1000.0000     0.9698  9182.0059
      8   C3 (   8)       9   H3 (   9)     1.0900   340.0000     1.0930     0.0031
      8   C3 (   8)      10   O3 (  10)     1.3700   320.0000     1.4362    

## Try with SYBYL atom types

In [11]:
host = load_mol2('generated/a-bam-p/MGO-sybyl.mol2')
components = split_topology(file_name='generated/a-bam-p/full.pdb')
host_topology = create_host_topology(components, host_resname='MGO')

In [12]:
host.GetMaxAtomIdx()

126

In [13]:
host_topology

<Structure 126 atoms; 6 residues; 132 bonds; NOT parametrized>

In [14]:
ff = ForceField('forcefield/smirnoff99Frosst.ffxml') 
system = ff.createSystem(host_topology.topology, [host],
                         nonbondedCutoff=1.1*unit.nanometer, 
                         ewaldErrorTolerance=1e-4
                         )


In [15]:
host_structure = pmd.openmm.topsystem.load_topology(host_topology.topology, system, host_topology.positions)

In [32]:
pmd.tools.printBonds(host_structure)

             Atom 1              Atom 2       R eq   Frc Cnst   Distance     Energy
      1   C1 (   1)       2   H1 (   2)     1.0900   340.0000     1.1077     0.1067
      1   C1 (   1)       3   O1 (   3)     4.0000  1000.0000     1.4676  6412.9044
      1   C1 (   1)       4   C2 (   4)     1.5260   310.0000     1.5259     0.0000
      1   C1 (   1)      16   O5 (  16)     4.0000  1000.0000     1.4738  6381.4472
      3   O1 (   3)      33   C4 (  33)     4.0000  1000.0000     1.4655  6423.7720
      4   C2 (   4)       5   H2 (   5)     1.0900   340.0000     1.0934     0.0040
      4   C2 (   4)       6   O2 (   6)     1.3700   320.0000     1.4372     1.4435
      4   C2 (   4)       8   C3 (   8)     1.5260   310.0000     1.5140     0.0446
      6   O2 (   6)       7  HO2 (   7)     4.0000  1000.0000     0.9698  9182.0059
      8   C3 (   8)       9   H3 (   9)     1.0900   340.0000     1.0930     0.0031
      8   C3 (   8)      10   O3 (  10)     1.3700   320.0000     1.4362    

In [33]:
for bond in host_structure.bonds:
    atom1, atom2 = bond.atom1, bond.atom2
    if bond.type is not None:
        if bond.type.req == 4.0000:
            print('%7d %4s (%4s) %7d %4s (%4s) %10.4f %10.4f' % (
            atom1.idx+1, atom1.name, atom1.type, atom2.idx+1,
            atom2.name, atom2.type, bond.type.req, bond.type.k)
            )


      1   C1 (   1)       3   O1 (   3)     4.0000  1000.0000
      1   C1 (   1)      16   O5 (  16)     4.0000  1000.0000
      3   O1 (   3)      33   C4 (  33)     4.0000  1000.0000
      6   O2 (   6)       7  HO2 (   7)     4.0000  1000.0000
     10   O3 (  10)      11  HO3 (  11)     4.0000  1000.0000
     12   C4 (  12)     108   O1 ( 108)     4.0000  1000.0000
     14   C5 (  14)      16   O5 (  16)     4.0000  1000.0000
     20   O6 (  20)      21  HO6 (  21)     4.0000  1000.0000
     22   C1 (  22)      24   O1 (  24)     4.0000  1000.0000
     22   C1 (  22)      37   O5 (  37)     4.0000  1000.0000
     24   O1 (  24)      54   C4 (  54)     4.0000  1000.0000
     27   O2 (  27)      28  HO2 (  28)     4.0000  1000.0000
     31   O3 (  31)      32  HO3 (  32)     4.0000  1000.0000
     35   C5 (  35)      37   O5 (  37)     4.0000  1000.0000
     41   O6 (  41)      42  HO6 (  42)     4.0000  1000.0000
     43   C1 (  43)      45   O1 (  45)     4.0000  1000.0000
     43 

In [31]:
for bond in host_structure.bonds:
    atom1, atom2 = bond.atom1, bond.atom2
    if bond.type is not None:
        if bond.type.req == 4.0000:
            print('%7d %4s (%4s) %7d %4s (%4s) %10.4f %10.4f' % (
            atom1.idx+1, atom1.name, atom1.type, atom2.idx+1,
            atom2.name, atom2.type, bond.type.req, bond.type.k)
            )


      1   C1 (   1)       3   O1 (   3)     4.0000  1000.0000
      1   C1 (   1)      16   O5 (  16)     4.0000  1000.0000
      3   O1 (   3)      33   C4 (  33)     4.0000  1000.0000
      6   O2 (   6)       7  HO2 (   7)     4.0000  1000.0000
     10   O3 (  10)      11  HO3 (  11)     4.0000  1000.0000
     12   C4 (  12)     108   O1 ( 108)     4.0000  1000.0000
     14   C5 (  14)      16   O5 (  16)     4.0000  1000.0000
     20   O6 (  20)      21  HO6 (  21)     4.0000  1000.0000
     22   C1 (  22)      24   O1 (  24)     4.0000  1000.0000
     22   C1 (  22)      37   O5 (  37)     4.0000  1000.0000
     24   O1 (  24)      54   C4 (  54)     4.0000  1000.0000
     27   O2 (  27)      28  HO2 (  28)     4.0000  1000.0000
     31   O3 (  31)      32  HO3 (  32)     4.0000  1000.0000
     35   C5 (  35)      37   O5 (  37)     4.0000  1000.0000
     41   O6 (  41)      42  HO6 (  42)     4.0000  1000.0000
     43   C1 (  43)      45   O1 (  45)     4.0000  1000.0000
     43 

## Visualize with `mdtraj`

In [18]:
import nglview, mdtraj

In [19]:
traj = mdtraj.load('generated/a-bam-p/MGO.mol2')

/home/dslochower/data/applications/anaconda3/envs/openforcefield/lib/python3.6/site-packages/pandas/io/parsers.py:2138: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/dslochower/data/applications/anaconda3/envs/openforcefield/lib/python3.6/site-packages/pandas/io/parsers.py:2140: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


In [20]:
view = nglview.show_mdtraj(traj)
view

A Jupyter Widget